# Justin Meyer        CS 5310 Ch. 4 Lab - Naive Bayes & kNN

In this lab I will do some data preprocessing to control for multicollinearity and then normalize the data set before running the kNN algorithm and also running the Naive Bayes model

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Import the Features data sets

df1 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/BG_features.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/FZ_features.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/MJ_features.csv')
df4 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/MR_features.csv')
df5 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/YH_features.csv')
Features_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [ ]:
# Import the Objects data sets

df1 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/BG_objects.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/FZ_objects.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/MJ_objects.csv')
df4 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/MR_objects.csv')
df5 = pd.read_csv('https://raw.githubusercontent.com/jdmeyer05/CS-5310/Ch-4/YH_objects.csv')
Objects_df = pd.concat([df1, df2, df3, df4, df5], axis=0)

In [ ]:
# Create the correlation matrix and convert values to absolute values

cor_output = Features_df.corr(method='pearson', min_periods=1).abs()

In [ ]:
# Create come logic to quickly remove all values correlated that is above .80

mask = np.triu(np.ones_like(cor_output, dtype=bool))
tri_df = cor_output.mask(mask)

to_drop = [c for c in tri_df.columns if any(tri_df[c] > 0.80)]

reduced_df = Features_df.drop(to_drop, axis=1)

In [ ]:
# Normalize the features using the preprocessing MinMaxScaler from sklearn

from sklearn import preprocessing
x = reduced_df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_df = pd.DataFrame(x_scaled)

In [ ]:
# Add back in the column headers

col_names = pd.DataFrame(reduced_df.columns)
normalized_df.columns = col_names

In [ ]:
# Fixes an issue with the indexing for the Objects dataframe

Objects_df.reset_index(drop=True, inplace=True)

In [ ]:
# Combine the dataframes

Full_df = Objects_df.join(normalized_df)
print(Full_df)

      Object  (theta,)  (alpha,)  ...  (fisher_info,)  (approx_entropy,)  (Hurst,)
0          3  0.177296  0.624308  ...        0.797799           0.799616  0.251382
1          2  0.375720  0.552349  ...        0.689945           0.650469  0.624338
2          5  0.483188  0.703620  ...        0.735459           0.810045  0.620844
3          1  0.482025  0.399041  ...        0.732483           0.790113  0.609107
4          1  0.749544  0.580101  ...        0.696438           0.712048  0.417715
...      ...       ...       ...  ...             ...                ...       ...
1265       2  0.445493  0.399895  ...        0.175358           0.098613  0.385846
1266       1  0.415657  0.337500  ...        0.165410           0.038583  0.573744
1267       2  0.177419  0.247819  ...        0.814458           0.811026  0.536741
1268       1  0.303140  0.558863  ...        0.852198           0.764204  0.257675
1269       3  0.217280  0.261221  ...        0.830967           0.838824  0.346550

[12

In [ ]:
# Split the data into features and labels

X = np.array(Full_df.drop(['Object'],1))
y = np.array(Full_df['Object'])

In [ ]:
# Split the data into training data set and testing data set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
# Train the kNN model and output the crosstab

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train, y_train)
nuro_test_pred = neigh.predict(X_test)
pd.crosstab(y_test, nuro_test_pred)

col_0,1,2,3,4,5
row_0,,,,,
1,26,11,3,6,9
2,20,14,7,6,9
3,16,9,10,5,11
4,13,7,9,9,8
5,18,9,7,5,7


In [ ]:
# How accurate is the kNN model?

accuracy = neigh.score(X_test, y_test)
print(accuracy)

0.25984251968503935


In [ ]:
# Split the data into training data set and testing data set

from sklearn.model_selection import train_test_split
X_trainN, X_testN, y_trainN, y_testN = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
mnb = MultinomialNB(alpha=1) 
Nuro_test_pred = mnb.fit(X_trainN, y_trainN).predict(X_testN) 
pd.crosstab(Nuro_test_pred, y_testN)

col_0,1,2,3,4,5
row_0,,,,,
2,0,0,0,0,1
3,5,1,4,4,7
4,1,6,1,5,1
5,49,49,46,37,37


In [ ]:
# How accurate is the Naive Bayes model?

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_testN, Nuro_test_pred)
print(cm)
accuracy_score(y_testN, Nuro_test_pred)

[[ 0  0  5  1 49]
 [ 0  0  1  6 49]
 [ 0  0  4  1 46]
 [ 0  0  4  5 37]
 [ 0  1  7  1 37]]


0.18110236220472442

# Conclusion

The kNN model was only 25.98% accurate in predicting which object belonged to which set of features. If we look at the confusion matrix for the Naive Bayes model the accuracy is only 18.11% which means this model is less accurate than using a kNN model.